In [1]:
import numpy as np

import astropy.io.fits as pyfits
from astropy.wcs import WCS
from astropy.coordinates import SkyCoord
from astropy.time import Time
from astropy import units as u

from sys import argv
import time
import os
import copy
import warnings

In [2]:
#---------------------------------------
# set the path for the forder
path_image='../Image_Data/'
path_BT = '../Image_BT/'
path_samping = '../Table_Data/'
path_figure = '../Plot_Figure/'
path_region = '../Plot_Region/'

In [3]:
#---------------------------------------
# set the outname
outname_b4       = 'n1333iras4a_B4_uv24_2900kl_sm200_100mas_noshift'
outname_b7       = 'n1333iras4a_B7_uv24_2900kl_sm200_100mas_noshift'
outname_mask     = 'n1333iras4a_B7_uv24_2900kl_sm200_100mas_noshift_mask_I'
outname_mask_4A1 = 'n1333iras4a_B7_uv24_2900kl_sm200_100mas_noshift_mask_4A1_I'
outname_mask_4A2 = 'n1333iras4a_B7_uv24_2900kl_sm200_100mas_noshift_mask_4A2_I'

#---------------------------------------
# set the output filename
def slt_imageName(outname):
    I_imageName          = '%s%s_I.image.fits'%(path_image, outname)
    Q_imageName          = '%s%s_Q.image.fits'%(path_image, outname)
    U_imageName          = '%s%s_U.image.fits'%(path_image, outname)
    PI_imageName         = '%s%s_PI.image.fits'%(path_image, outname)
    PI_cut3PI_imageName  = '%s%s_PI_cut3PI.image.fits'%(path_image, outname)
    PA_imageName         = '%s%s_PA.image.fits'%(path_image, outname)
    Per_imageName        = '%s%s_Per.image.fits'%(path_image, outname)
    Per_cut3PI_imageName = '%s%s_Per_cut3PI.image.fits'%(path_image, outname)
    BT_imageName         = '%s%s_BT.fits'%(path_BT, outname) 
    
    return I_imageName, Q_imageName, U_imageName, PI_imageName, PI_cut3PI_imageName,\
            PA_imageName, Per_imageName, Per_cut3PI_imageName, BT_imageName
    

BT_B4_imageName      = '%s%s_BT.fits'%(path_BT, outname_b4)
BT_B7_imageName      = '%s%s_BT.fits'%(path_BT, outname_b7)
BT_slt_imageName     = '%s%s_BT.fits'%(path_BT, outname_b4)
BT_div_imageName     = '%sBT_B4_div_BT_B7.fits'%(path_BT)
Mask_imageName       = '%s%s.image.fits'%(path_image, outname_mask)
Mask_4A1_imageName   = '%s%s.image.fits'%(path_image, outname_mask_4A1)
Mask_4A2_imageName   = '%s%s.image.fits'%(path_image, outname_mask_4A2)

#----
BT_B4            = '%s%s_BT.fits'%(path_BT, outname_b4)
BT_B7            = '%s%s_BT.fits'%(path_BT, outname_b7)
BT_slt_image     = '%s%s_BT.fits'%(path_BT, outname_b4)
BT_div_image     = '%sBT_B4_div_BT_B7.fits'%(path_BT)
Mask_image       = '%s%s.image.fits'%(path_image, outname_mask)
Mask_4A1_image   = '%s%s.image.fits'%(path_image, outname_mask_4A1)
Mask_4A2_image   = '%s%s.image.fits'%(path_image, outname_mask_4A2)

In [4]:
#---------------------------------------
# set the 
dist = 293 # [pc]
sampling_all = 10

ra_4A1 = 52.2939125        # RA  [deg], central of 4A1
dec_4A1 = 31.2252611       # Dec [deg], central of 4A1
ra_4A2 = 52.2934625        # RA  [deg], central of 4A2
dec_4A2 = 31.2255806       # Dec [deg], central of 4A2
ra_4A1_4A2 = 52.2937125    # RA  [deg], central of 4A
dec_4A1_4A2 = 31.2255431   # Dec [deg], central of 4A

Coord_4A = SkyCoord('03h29m10.498s', '+31d13m31.293s', frame='icrs')
Coord_4A1 = SkyCoord('03h29m10.538s', '+31d13m30.890s', frame='icrs')
Coord_4A2 = SkyCoord('03h29m10.431s', '+31d13m32.04s', frame='icrs')



In [5]:
#---------------------------------------
# set the rms and criteria
# ALMA Band 4
rms_I_b4       = 1.0e-4              #(Jy beam-1)
rms_Q_b4       = 1.2e-5              #(Jy beam-1)
rms_PI_b4      = 1.2e-5              #(Jy beam-1)
I_clip_b4      = 3*rms_I_b4          #(Jy beam-1)
PI_lowclip_b4  = 3*rms_PI_b4         #(Jy beam-1)
PI_highclip_b4 = 1e99*rms_PI_b4      #(Jy beam-1)
sampling_b4    = sampling_all
PI_vmax_b4     = 2.849e-3            #(Jy beam-1)
convert_unit_b4 = 'Jy2mJy'

# ALMA Band 7
rms_I_b7       = 4.5e-4              #(Jy beam-1)
rms_Q_b7       = 3.0e-5              #(Jy beam-1)
rms_PI_b7      = 3.0e-5              #(Jy beam-1)
I_clip_b7      = 3*rms_I_b7          #(Jy beam-1)
PI_lowclip_b7  = 3*rms_PI_b7         #(Jy beam-1)
PI_highclip_b7 = 1e99*rms_PI_b7      #(Jy beam-1)
sampling_b7    = sampling_all
PI_vmax_b7     = 1.005e-2            #(Jy beam-1)
convert_unit_b7 = 'Jy2mJy'

# function for select parameter
def slt_parm(outname):
    
    if outname == outname_b4:
        rms_I=rms_I_b4
        rms_Q=rms_Q_b4
        rms_PI=rms_PI_b4
        I_clip=I_clip_b4
        PI_lowclip=PI_lowclip_b4
        PI_highclip=PI_highclip_b4
        sampling=sampling_b4
        PI_vmax=PI_vmax_b4
        convert_unit=convert_unit_b4
    elif outname == outname_b7:
        rms_I=rms_I_b7
        rms_Q=rms_Q_b7
        rms_PI=rms_PI_b7
        I_clip=I_clip_b7
        PI_lowclip=PI_lowclip_b7
        PI_highclip=PI_highclip_b7
        sampling=sampling_b7
        PI_vmax=PI_vmax_b7
        convert_unit=convert_unit_b7
        
    return rms_I, rms_Q, rms_PI, I_clip, PI_lowclip, PI_highclip, \
            sampling, PI_vmax, convert_unit

In [6]:
# ALMA Band 4
scale_10percent_b4  = 0.1e-2
scale_line_b4       = 0.006
seg_color_b4        = 'blue'
seg_width_b4        = 2
seg_color_base_b4   = 'white'
seg_width_base_b4   = 3
    
# ALMA Band 7
scale_10percent_b7  = 0.1e-2
scale_line_b7       = 0.008
seg_color_b7        = 'black'
seg_width_b7        = 4
seg_color_base_b7   = 'white'
seg_width_base_b7   = 5

# function for select parameter
def slt_seg(outname):
    if outname == outname_b4:
        scale_10percent=scale_10percent_b4
        scale_line=scale_line_b4
        seg_color=seg_color_b4
        seg_width=seg_width_b4
        seg_color_base=seg_color_base_b4
        seg_width_base=seg_width_base_b4
    elif outname == outname_b7:
        scale_10percent=scale_10percent_b7
        scale_line=scale_line_b7
        seg_color=seg_color_b7
        seg_width=seg_width_b7
        seg_color_base=seg_color_base_b7
        seg_width_base=seg_width_base_b7
        
    return scale_10percent, scale_line, seg_color, seg_width, \
            seg_color_base, seg_width_base

In [7]:
def slt_plotName(outname, reg_parm):
    
    output_E_reg_name = '%s%s_%s_E_PA.reg'%(path_region, outname, reg_parm)
    output_E_reg_base_name = '%s%s_%s_E_PA_base.reg'%(path_region, outname, reg_parm)
    output_B_reg_name = '%s%s_%s_B_PA.reg'%(path_region, outname, reg_parm)
    output_B_reg_base_name = '%s%s_%s_B_PA_base.reg'%(path_region, outname, reg_parm)
    
    return output_E_reg_name, output_E_reg_base_name, \
            output_B_reg_name, output_B_reg_base_name
    

In [8]:
# convert the unit from milliarcsecond to radius
def mas2rad(u_mas):
    return np.deg2rad(np.divide(u_mas, 36e5))

# convert the unit from milliarcsecond to deg
def mas2dec(u_mas):
    return np.divide(u_mas, 36e5)

# convert the unit from deg to milliarcsecond
def deg2mas(u_deg):
    return u_deg*36e5

In [9]:
# get the header
outname = outname_b4
I_image = '%s%s_I.image.fits'%(path_image, outname)
I_data = pyfits.getdata(I_image)
I_hd = pyfits.getheader(I_image)

# get the parameter from the header
hd_freq = I_hd['RESTFRQ']           # frequency [Hz]
hd_bmaj = I_hd['BMAJ']              # [deg]
hd_bmin = I_hd['BMIN']              # [deg]
hd_bpa = I_hd['BPA']                # [deg]

hd_cell = np.abs(I_hd['CDELT1'])    # cell size [deg]
hd_ra = I_hd['CRVAL1']              # ra [deg]
hd_dec = I_hd['CRVAL2']             # dec [deg]
hd_date = I_hd['DATE']              # date [y-m-dTh:m:s]
hd_pixelNum = I_hd['NAXIS1']        # pixel number per axis [pixel]
hd_unit = I_hd['BUNIT']

# calculate the sampling number
sampling = np.divide(hd_bmin, hd_cell*2)
sampling_int = np.around(sampling)

# print the results
#print sampling_int

#print hd_unit

In [10]:
# print I_image
# im_unit = pyfits.getval(I_image, 'BUNIT')
# print im_unit

# pyfits.setval(I_image, 'BUNIT', value = 'Jy/beam')

# im_unit = pyfits.getval(I_image, 'BUNIT')
# print im_unit

In [11]:
'''
print hd_freq
print hd_bmaj, hd_bmin, hd_bpa
print hd_bmin*3600
print hd_cell*3600
sampling = np.divide(hd_bmin, hd_cell*2)
sampling_int = np.around(sampling)
print sampling
print sampling_int
print hd_pixelNum
print hd_ra, hd_dec
print hd_date
'''

'\nprint hd_freq\nprint hd_bmaj, hd_bmin, hd_bpa\nprint hd_bmin*3600\nprint hd_cell*3600\nsampling = np.divide(hd_bmin, hd_cell*2)\nsampling_int = np.around(sampling)\nprint sampling\nprint sampling_int\nprint hd_pixelNum\nprint hd_ra, hd_dec\nprint hd_date\n'

In [12]:
Coord_4A = SkyCoord('03h29m10.498s', '+31d13m31.293s', frame='icrs')
Coord_4A1 = SkyCoord('03h29m10.538s', '+31d13m30.890s', frame='icrs')
Coord_4A2 = SkyCoord('03h29m10.431s', '+31d13m32.04s', frame='icrs')
Coord_4A_ra_deg = Coord_4A.ra.deg
Coord_4A_dec_deg = Coord_4A.dec.deg
Coord_4A1_ra_deg = Coord_4A1.ra.deg
Coord_4A1_dec_deg = Coord_4A1.dec.deg
Coord_4A2_ra_deg = Coord_4A2.ra.deg
Coord_4A2_dec_deg = Coord_4A2.dec.deg

In [13]:
'''
# Choose colormap
cmap = plt.cm.gist_rainbow_r
# Get the colormap colors
my_cmap = cmap(np.arange(cmap.N))
# Set alpha
my_cmap[:,-1] = np.linspace(0.8, 0.3, cmap.N)
# Create new colormap
my_cmap = ListedColormap(my_cmap)
'''

'\n# Choose colormap\ncmap = plt.cm.gist_rainbow_r\n# Get the colormap colors\nmy_cmap = cmap(np.arange(cmap.N))\n# Set alpha\nmy_cmap[:,-1] = np.linspace(0.8, 0.3, cmap.N)\n# Create new colormap\nmy_cmap = ListedColormap(my_cmap)\n'